In [ ]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import sys
import os

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

conf = SparkConf().setAppName("Spark - RDD par")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName("Spark - RDD par").getOrCreate()

In [ ]:
import os
def remove_file(file):
    if os.path.exists(file):
        os.remove(file)
remove_file("ign.csv")

In [ ]:
import requests
r = requests.get("https://jankiewicz.pl/bigdata/bigdata-sp/ign.csv", allow_redirects=True)
open('ign.csv', 'wb').write(r.content)

2019628

In [ ]:
rawIgn = sc.textFile("ign.csv")
rawIgn.count()

18626

In [ ]:
rawIgn.first()

',score_phrase,title,url,platform,score,genre,editors_choice,release_year,release_month,release_day'

In [ ]:
import re
tabIgn = rawIgn.map(lambda line: re.split(",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)",line))
tabIgn.count()

18626

In [ ]:
tabIgn.first()

['',
 'score_phrase',
 'title',
 'url',
 'platform',
 'score',
 'genre',
 'editors_choice',
 'release_year',
 'release_month',
 'release_day']

In [ ]:
PLATFORM = 4
SCORE = 5
GENRE = 6
RELEASE_YEAR = 8

In [ ]:
gameInfosRdd = tabIgn.filter(lambda tab: len(tab)== 11 and len(tab[0])>0)

In [ ]:
gameInfosRdd.first()

['0',
 'Amazing',
 'LittleBigPlanet PS Vita',
 '/games/littlebigplanet-vita/vita-98907',
 'PlayStation Vita',
 '9.0',
 'Platformer',
 'Y',
 '2012',
 '9',
 '12']

In [18]:
# jakiego typu są g1, g2 i g3?
g1 = gameInfosRdd.groupBy(lambda gi: gi[GENRE])
type(g1)

pyspark.rdd.PipelinedRDD

In [15]:
g2 = gameInfosRdd.map(lambda gi: (gi[GENRE],gi))

pyspark.rdd.PipelinedRDD

In [ ]:
g3 = gameInfosRdd.keyBy(lambda gi: gi[GENRE]) # genre

In [19]:
# Co wyliczają poniższe wyrażenia?
# Jakiego typu są wyniki?
# Czy wszystkie dają ten sam wynik?
g4 = gameInfosRdd.keyBy(lambda gi: gi[GENRE]).aggregateByKey(0.0, lambda m,gi : m + float(gi[SCORE]), lambda mx,my: mx + my)

In [20]:
g5 = gameInfosRdd.map(lambda gi: (gi[GENRE],gi)).groupByKey().\
mapValues(lambda gis: sum(list(map(lambda x:float(x[SCORE]), gis))))

In [21]:
g6 = gameInfosRdd.map(lambda gi: (gi[GENRE],float(gi[SCORE]))).reduceByKey(lambda mx,my: mx + my)

In [22]:
%%time
g4.first()

CPU times: total: 0 ns
Wall time: 2.48 s


('Platformer', 5914.5000000000055)

In [23]:
%%time
g5.first()

CPU times: total: 15.6 ms
Wall time: 2.51 s


('Platformer', 5914.5000000000055)

In [24]:
%%time
g6.first()

CPU times: total: 0 ns
Wall time: 2.25 s


('Platformer', 5914.5000000000055)

In [25]:
g7 = gameInfosRdd.map(lambda gi: (gi[GENRE],gi)).groupByKey().\
mapValues(lambda gis: sum(list(map(lambda x:float(x[SCORE]), gis))))

In [26]:
g8 = gameInfosRdd.keyBy(lambda gi: gi[GENRE]).countByKey()
{k: g8[k] for k in list(g8)[:10]}

{'Platformer': 823,
 'Puzzle': 776,
 'Sports': 1916,
 'Strategy': 1071,
 'Fighting': 547,
 'RPG': 980,
 '': 36,
 '"Action, Adventure"': 765,
 'Adventure': 1175,
 'Action': 3797}